In [1]:
from ezc3d import c3d
import pandas as pd
from loguru import logger
import os
import numpy as np
import math
from IPython.core.debugger import set_trace

In [2]:
path_to_study = r"/mnt/S/2202000565 - Fencing/data_fsp01/" #'S:\\2202000565 - Fencing\\data_fsp01\\'


In [3]:
## c3d files are generated by exporting the TAK files in OptiTrack Motive, using the export to c3d option
## We export into the same 'ot' folder as the TAK files
## Units are mmlogger
## Axis convention is Visual3d/MotionMonitor
## Options that are on are
## - Use zero-based frame index
## - Unlabeled markers
## - Timecode
c3d_files = []

# Traverse the directory tree using os.walk()
for root, dirs, files in os.walk(path_to_study):
    for file in files:
        # Check if the file has a .c3d extension
        if file.endswith('.c3d'):
            # Add the file path to the list of .c3d files
            c3d_files.append(os.path.join(root, file))

In [4]:
display(c3d_files)

['/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_001.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_002.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_003.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_004.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_005.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_006.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_EnGarde_007.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_FeintDisengageLunge_012.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_FootworkPlusLunge_008.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_FootworkPlusLunge_009.c3d',
 '/mnt/S/2202000565 - Fencing/data_fsp01/20220701_01_001/ot/se001_FootworkPlusLunge_010.c3d',
 '/mnt/S/2202000565 - Fencing/data

In [18]:
## The following code is just to help parse the file 'fencing snapshots data.xlsx' into a pandas dataframe.
## The code is particular to the structure of this file and will not work for other files.
labels = pd.read_excel(r"fencing snapshots data.xlsx", header=0)

In [19]:
# Drop all rows after index 97
labels = labels.drop(labels.index[97+1:])

# Fill the session names through the column
labels['session'] = labels['session'].fillna(method='ffill')

# In the file, the first actions are labeled by column 'action 1 start' and 'action 1 end'
# For consistency, we will rename these columns to '1s' and '1e' to match the format for all other actions
labels = labels.rename(columns={'action 1 start': '1s', 'action 1 end': '1e', 'action 1 type': '1t', '50y': '50t', '84y': '84t'})

In [7]:
#display(labels.iloc[3:20, :20])

In [8]:
dataset_name = 'fsp01'

save_path = os.path.join(os.getcwd(), 'soma-root', 'support_files/evaluation_mocaps/original', dataset_name)
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [9]:
## Go through each c3d file, and split it into individual c3d files for each action in the session
## Splitting frame numbers come from the labels dataframe

def get_labels_for_file(c3d_file: str) -> pd.Series:
    """
    Takes in a full path to a c3d file.
    Returns a Pandas series which contains the action names and start/end frames for every action in the file.
    """
    # The directory name matches the session name of the labels dataframe
    session = os.path.basename(os.path.dirname(os.path.dirname(c3d_file)))
    file = os.path.basename(c3d_file)
    # print(session)

    # Get the 'vidName': e.g., for basename se001_EnGarde_001.c3d, vidName is EnGarde_001
    # vidName is used to index into the labels dataframe
    vidName = file[file.index("_")+1:file.index(".c3d")]
    # print(vidName)

    # Create a mask to find the rows which match both the session and vid criteria
    # (should only be one row which matches)
    mask = labels['session'].str.contains(session) & (labels['vid'] == vidName)
    if mask.sum() > 1:
        logger.warning("More than one match found for session {} and vid {}".format(session, vidName))
    elif mask.sum() == 0:
        logger.error("No match found for session {} and vid {}".format(session, vidName))
        raise FileNotFoundError
    
    row = labels[mask].iloc[0]
    return row



In [15]:
def save_action_c3d(action_type: str,
                 session: str,
                 file: str,
                 c: c3d,
                 labels: pd.Series,
                 ix: int,
                 orig_point_data: np.ndarray) -> None:
    """
    For a given file and specific action index, save a new c3d file containing only the data for that action.
    Returns 1 if file is erroring upon write, else 0.
    """

    action_start = int(labels.loc[f"{ix}s"])
    action_end = int(labels.loc[f"{ix}e"])

    type_name = ''

    # sort action type 
    if "lunge" in action_type or "lunging" in action_type:
        if "RH" in action_type:
            type_name = "lunge_RH"
        elif "LH" in action_type:
            type_name = "lunge_LH"
    elif "entire motion engarde" in action_type:
        if "RH" in action_type:
            type_name = "engarde_RH"
        elif "LH" in action_type:
            type_name = "engarde_LH"
    
    if len(type_name) < 1:
        # our current filters don't catch all action types, so we log them here
        logger.warning(f"discovered non-standard action type: {action_type} at index {ix}")
        return
    
    # logger.debug(f"type_name: {type_name}")
    arr_action = orig_point_data[:, :, action_start:action_end+1]
    
    if arr_action.shape[2] != action_end - action_start + 1:
        logger.error(f"error in action {ix} for file {file}: check indeces in labels file")
        return

    # set c3d point data to just be from our action and save as a new file
    # note that for each action we just keep rewriting the same file with new point data
    c['data']['points'] = arr_action

    out_fname = f"{file[:-4]}_{type_name}_action{ix}.c3d"
    out_dir = os.path.join(save_path, session)

    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    # we delete the meta points from before since new meta points are auto-generated when saving
    del c['data']['meta_points']

    outpath = os.path.join(out_dir, out_fname)

    logger.debug(f"saving file: {outpath}")
    try:
        c.write(outpath)
        return 0
    except Exception as e:
        logger.error(f"failed to write file: {outpath}")
        logger.error(e)
        return 1

def split_c3d(c3d_file: str, labels: pd.Series) -> list:
    """
    @inputs:
    - full path to c3d file
    - pd.Series containing labels for the start/end frames of each action in the c3d take
    @outputs:
    - generates new c3d files for each action and saves them in the given directory
    """
    try:
        c = c3d(c3d_file)
    except ValueError:
        logger.error(f"Failed to read c3d file: {c3d_file}")
        return
    
    logger.debug(f"number of marker points\t {c['parameters']['POINT']['USED']['value'][0]}");  # Print the number of marker-points used
    orig_point_data = c['data']['points']

    # session name is the date + the subjectid
    session = os.path.basename(os.path.dirname(os.path.dirname(c3d_file)))
    file = os.path.basename(c3d_file)


    # get total number of actions in the file
    ix = 1
    while True:
        col = f"{ix}t"
        action_type = labels.loc[col] # action description as string
        # logger.debug(f"action_type: {action_type}")
        if type(action_type) is not str and math.isnan(action_type):
            logger.debug(f"Done with file, no more actions. The file {c3d_file} has {ix-1} actions")
            break

        # When code is 1, we have an error writing the file
        code = save_action_c3d(action_type, session, file, c, labels, ix, orig_point_data)
        if code == 1:
            return
        ix+=1


In [20]:
logger.add("logs/split_c3d.log", rotation="500 MB", compression="zip")
for c3d_file in c3d_files[18:]:
    logger.debug(f"Processing file {c3d_file}")
    file_labels = get_labels_for_file(c3d_file)
    split_c3d(c3d_file, file_labels)

2023-04-25 02:54:29.250 | DEBUG    | __main__:<module>:2 - processing file /mnt/S/2202000565 - Fencing/data_fsp01/20220708_01_002/ot/se002_FeintDisengageLunge_011.c3d
2023-04-25 02:54:36.043 | DEBUG    | __main__:split_c3d:80 - number of marker points	 137
2023-04-25 02:54:36.045 | WARNING  | __main__:save_action_c3d:32 - discovered non-standard action type: proper engarde 1 LH at index 1
2023-04-25 02:54:36.046 | WARNING  | __main__:save_action_c3d:32 - discovered non-standard action type: feint 1 LH at index 2
2023-04-25 02:54:36.049 | DEBUG    | __main__:save_action_c3d:57 - saving file: /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/fsp01/20220708_01_002/se002_FeintDisengageLunge_011_lunge_LH_action3.c3d
2023-04-25 02:54:36.067 | DEBUG    | __main__:save_action_c3d:57 - saving file: /home/ritaank/Documents/dev/SOMA-interface/soma-root/support_files/evaluation_mocaps/original/fsp01/20220708_01_002/se002_FeintDisengageLunge_011_lunge_LH_

In [ ]:
c = c3d(filename)
print(c['parameters']['POINT']['USED']['value'][0]);  # Print the number of marker-points used
point_data = c['data']['points']

NameError: name 'filename' is not defined

print(5)

In [ ]:
point_data.shape

(4, 45, 19272)

In [ ]:
arr_half = point_data[:, :, :point_data.shape[-1]//2]
arr_half.shape

(4, 45, 9636)

In [ ]:
c['data']['points'] = arr_half

In [ ]:
print(f"firsthalf_{filename[:-4]}.c3d")

firsthalf_se001_FootworkPlusLunge_010.c3d


In [ ]:
del c['data']['meta_points']
c.write(f"firsthalf_{filename[:-4]}.c3d")

In [ ]:
# end 

In [ ]:
labels = pd.read_csv("fencing snapshots data.csv", header=0)

In [ ]:
c3d2 = c3d()
c3d2['parameters'] = c['parameters']

TypeError: 'c3d' object does not support item assignment